# The Dealer class

This notebook provides some documentation about Dealer objects.

## Importing the Dealer class

The current working directory needs to be "fivecarddraw" in order to run the code cells of this notebook. Assuming the fivecarddraw repository has been cloned, then this notebook is contained within the "fivecarddraw/notebooks" folder. To change the current working directory, run the next cell.

In [2]:
from os import chdir, getcwd

if not getcwd().endswith("fivecarddraw"):
    chdir("..")
    
print(f"Current Directory: {getcwd()}")

Current Directory: d:\My Projects\Python\Games\fivecarddraw


Then the Dealer class can be imported in the usual pythonic way.

In [40]:
from fivecarddraw import Dealer

## Creating a Dealer

The Dealer class is a subclass of the Table class, which is essentially a list of Player objects, with a Deck and a pot. The Dealer class can therefore access any gamestate related data, to perform any actions it needs to. A consequence of the inheritence is that Dealers need to be initialised with a list of Player objects.  

In [41]:
from fivecarddraw import Player

players = [Player(f"Player {i+1}") for i in range(5)]
print(f"Players: {players}")

Players: [Player 1, Player 2, Player 3, Player 4, Player 5]


Initialising the dealer will take a moment as the dealer object will populate 3 quite large lookup tables. The lookup tables play a part in hand evaluation, which is outlined in more detail in the hand.ipynb notebook.

In [42]:
dealer = Dealer(players)

print(f"Players: {dealer}")
print(f"Deck: {dealer.deck}")

Players: [Player 1, Player 2, Player 3, Player 4, Player 5]
Deck: [2♠, 3♠, 4♠, 5♠, 6♠, 7♠, 8♠, 9♠, 10♠, J♠, Q♠, K♠, A♠, 2♡, 3♡, 4♡, 5♡, 6♡, 7♡, 8♡, 9♡, 10♡, J♡, Q♡, K♡, A♡, 2♢, 3♢, 4♢, 5♢, 6♢, 7♢, 8♢, 9♢, 10♢, J♢, Q♢, K♢, A♢, 2♣, 3♣, 4♣, 5♣, 6♣, 7♣, 8♣, 9♣, 10♣, J♣, Q♣, K♣, A♣]


## Tracking the Table Button 

The table button is tracked by maintaining a sequence of players, that mimics player seat positions clockwise around a table; and cycling the sequence, so that the player who has the button is last in the sequence. This means that turn order begins with the player at index 0 of the dealer.

In [43]:
dealer.MoveButton()
print(f"Players: {dealer}")
print(f"Button player: {dealer[-1]}")

Players: [Player 2, Player 3, Player 4, Player 5, Player 1]
Button player: Player 1


## Dealing, Evaluating and Collecting Hands

The way that Dealers deal hands mimics the way that casino dealers deal hands at casinos. The deck is shuffled. Then a card is given to each player starting from index 0, and then another card is dealt to each 4 more times. Evaluation and collection is outlined in the hand.ipynb notebook. 

In [44]:
dealer.ShuffleDeck()
dealer.DealHands()
rankings = dealer.EvaluateHands()

for i in range(len(dealer)):
    rating, hit = rankings[dealer[i].name], dealer.Lookup(rankings[dealer[i].name])
    print(f"{dealer[i]}'s hand: {dealer[i].hand} (The number {rating} best possible hand with {hit}.)")

dealer.CollectCards()

Player 2's hand: 8♢ A♠ J♣ 10♣ 10♢ (The number 4226 best possible hand with pair.)
Player 3's hand: 8♡ 4♡ K♣ 10♡ 3♠ (The number 6915 best possible hand with high card.)
Player 4's hand: 2♢ K♡ 7♢ A♡ 6♢ (The number 6333 best possible hand with high card.)
Player 5's hand: J♠ 9♡ 6♣ 6♡ 5♣ (The number 5231 best possible hand with pair.)
Player 1's hand: 9♢ 4♠ 9♠ K♠ Q♡ (The number 4487 best possible hand with pair.)


## Taking Bets

The Dealer class is given the responsibility to take bets from players and put them in the pot, and this is mainly to ensure that the bets are legal, and are communicated to everybody. The semantics of this process might look a little bit off, but the gameplay loop itself is a subclass of the Dealer class, so when these methods are used in an actual game by the FiveCardDraw object, the semantics make more sense. 

In [45]:
for player in dealer:
    player.chips += 100

dealer.ante = 10
dealer.TakeAnte()


Player 2 paid 10 chips for the ante.
Player 3 paid 10 chips for the ante.
Player 4 paid 10 chips for the ante.
Player 5 paid 10 chips for the ante.
Player 1 paid 10 chips for the ante.


The TakeBet() method not only adds chips to a pot, it also categorises an action depending on how many chips are in the pot.

In [47]:
for player in dealer:
    player.chips += 100

dealer.TakeBet(dealer[0], 0)
dealer.TakeBet(dealer[1], 10)
dealer.TakeBet(dealer[2], 10)
dealer.TakeBet(dealer[3], 0)
dealer.TakeBet(dealer[4], dealer[4].chips)

for player in dealer:
    dealer.pot[player.name] = 0

Player 2 has checked.
Player 3 has raised by 10.
Player 4 has called.
Player 5 has folded.
Player 1 has raised by 90 and gone allin!


The TakeBet() method also updates Player attributes which track whether the player has folded, min-called, or has gone all-in. This helps control the gameplay loop during FiveCardDraw.BettingPhase(). The players states are treated as mutually exclusive. 

In [48]:
for player in dealer:
    print(f"{player} has mincalled: {player.has_mincalled}   ", f"{player} has allin: {player.has_allin}   ", f"{player} has folded: {player.has_mincalled}")

Player 2 has mincalled: False    Player 2 has allin: False    Player 2 has folded: False
Player 3 has mincalled: False    Player 3 has allin: False    Player 3 has folded: False
Player 4 has mincalled: False    Player 4 has allin: False    Player 4 has folded: False
Player 5 has mincalled: False    Player 5 has allin: False    Player 5 has folded: False
Player 1 has mincalled: False    Player 1 has allin: True    Player 1 has folded: False


Validation of a bet amount is currently performed by the FiveCardDraw object, although this may change in the future. For more information about the FiveCardDraw object, see the Simulation.ipynb notebook.

## Rewarding Players

Rewarding players involves a few different stages. The first step is to evaluate the hands of all of the players. 

### Evaluating Hands

In [5]:
from fivecarddraw import Dealer, Player

players = [Player(f"Player {i+1}") for i in range(5)]
dealer = Dealer(players)

dealer.ShuffleDeck()
dealer.DealHands()

ratings = dealer.EvaluateHands()
for i in range(len(dealer)):
    rating, hit = ratings[dealer[i].name], dealer.Lookup(ratings[dealer[i].name])
    print(f"{dealer[i]}'s hand: {dealer[i].hand} (The number {rating} best possible hand with {hit}.)")



Player 1's hand: 8♠ 2♠ 3♡ 7♢ K♡ (The number 6982 best possible hand with high card.)
Player 2's hand: 2♡ J♣ Q♠ 10♣ J♠ (The number 4093 best possible hand with pair.)
Player 3's hand: 9♠ 4♣ A♢ 3♠ 3♢ (The number 5784 best possible hand with pair.)
Player 4's hand: K♢ 9♡ 8♣ A♠ 9♣ (The number 4429 best possible hand with pair.)
Player 5's hand: 6♢ K♣ 8♡ Q♣ J♡ (The number 6694 best possible hand with high card.)


### Deciding Reward Priority

The next stage is to determine which players haven't folded, and sort them by their hand strength. This creates a priority list of what order to determine the rewards of players. Rewards are determined for players with stronger hands first.   

In [8]:
for i in range(len(dealer)):
    dealer[i].has_allin = True
    dealer.pot[dealer[i].name] = 100

print(f"Total chips in pot: {sum(dealer.pot.values())}")

players = [player for player in dealer if not player.has_folded]
players.sort(key = lambda x: (ratings[x.name], dealer.pot[x.name]))

print(f"Players in order of strength of hand: {[(player, ratings[player.name]) for player in players]}")



Total chips in pot: 500
Players in order of strength of hand: [(Player 2, 4093), (Player 4, 4429), (Player 3, 5784), (Player 5, 6694), (Player 1, 6982)]


### Calculating Rewards

Rewards are then calculated and given to each player in sequence, until no more chips remain in the pot.

#### Handling Side Pots

To account for side pots, dealer.pot is actually a dict with player.names as keys. The maximum a player can take from the pot is capped by how much they have contributed to the pot. They cannot take more than that much out of any other players contribution.

In [9]:
print(f"Number of side pots: {len(set(dealer.pot.values()))-1}")

Number of side pots: 0


#### Handling Split Pots

To account for split pots, the groupby function of itertools is used to group players who have exactly the same ranked hands.

In [11]:
from itertools import groupby

splits =  [list(players) for _, players in groupby(iterable = players, key = lambda x : ratings[x.name])]

print(f"Players grouped by the strength of their hands: {splits}")
print(f"Number of players with the same ranked hands: {len(players) - len(splits)}")

Players grouped by the strength of their hands: [[Player 2], [Player 4], [Player 3], [Player 5], [Player 1]]
Number of players with the same ranked hands: 0


If players share the same strength hand, and a reward is to be distributed to them, then the players are sorted by how much they have contributed. Excess rewards are claimed and shared between players who have paid excess. Remaining rewards are shared between everybody. 

In [12]:
contributions = {"person1": 100, "person2" : 150, "person3" : 200, "person4" : 150, "person5" : 200, "folded1" : 100, "folded2" : 150}
people = sorted(["person1", "person2", "person3", "person4", "person5"], key = lambda x : contributions[x])
print(f"All Contributors: {contributions}")
print(f"Winners: {[(person, contributions[person]) for person in people]}")

total = 0 
for i, winner in enumerate(people):
    if any(contributions.values()):
        contributers = sorted(contributions.keys(), key = lambda x: contributions[x], reverse = True)
    
    for contributer in contributers:
        if not contributions[contributer]:
            break
        if contributer == winner:
            continue
        
        if contributions[contributer] > contributions[winner]:
            total += contributions[winner]
            contributions[contributer] -= contributions[winner]
        else:
            total += contributions[contributer]
            contributions[contributer] = 0
    
    total += contributions[winner]
    contributions[winner] = 0
    
    reward = total // (len(people) - i)
    total -= reward
    
    if reward:
        print(f"{winner} won {reward} chips.")

All Contributors: {'person1': 100, 'person2': 150, 'person3': 200, 'person4': 150, 'person5': 200, 'folded1': 100, 'folded2': 150}
Winners: [('person1', 100), ('person2', 150), ('person4', 150), ('person3', 200), ('person5', 200)]
person1 won 140 chips.
person2 won 202 chips.
person4 won 202 chips.
person3 won 253 chips.
person5 won 253 chips.


### Paying Players

Next, rewards are paid directly to players, and information about player rewards are printed. The hands of players are revealed in turn order, unless a player holds a weaker hand than a hand that is already revealed.

In [13]:
print(f"Chips in the pot {sum(dealer.pot.values())}")
dealer.Payout()

Chips in the pot 500
Player 1 mucked with 8♠ 2♠ 3♡ 7♢ K♡ (high card)
Player 2 won 500 chips with 2♡ J♣ Q♠ 10♣ J♠ (pair)
Player 3 mucked.
Player 4 mucked.
Player 5 mucked.
